In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from pathlib import Path
from run import parse_arguments, configure, extract_flight_data, find_all_matching_pairs, select_matching_pairs, gather_metadata, image_processing_engine, analyzis, auto_angles_offset_pre_computation
from utils.utils_IRdrone import add_offset_theoretical_angles
# %matplotlib notebook
%matplotlib inline



In [ ]:
# --------------------------------------------------------------------------------------------------------------
config_file_path = r"D:\Air-Mission\FLY-test-GITHUB\config.json"
# --------------------------------------------------------------------------------------------------------------

In [ ]:
# 0 > Parse arguments, prepare configuration dictionary
# --------------------------------------------------------------------------------------------------------------
assert Path(config_file_path).exists(), "please provide the path to the configuration .json file"
arguments = []
arguments+= ["--config", config_file_path]
arguments+= ["--selection", "best-mapping"]
arguments+= ["--offset", "auto"]
args = parse_arguments(arguments)
conf = configure(args)



In [ ]:
# 1 > Extraction of flight data
# --------------------------------------------------------------------------------------------------------------
flight_data_dict = extract_flight_data(conf["config_file"])
conf["out_images_folder"] = flight_data_dict["out_images_folder"]
conf["timelapse_vis_interval"] = flight_data_dict["timelapse_vis_interval"]
if conf["timelapse_vis_interval"] <= 0:
    conf["selection"] = "all"
flight_data_dict.keys()

In [ ]:
# manually define your offset here!
# flight_data_dict["offset_angles"] = [0., 0., 0.]

In [ ]:
# 2 > Find all matching pairs of images from both cameras
# 3 > Then retrieve metadata from exifs & altitudes
# --------------------------------------------------------------------------------------------------------------
shooting_pts_list = find_all_matching_pairs(flight_data_dict)
gather_metadata(flight_data_dict, shooting_pts_list, conf)
print(shooting_pts_list[0])

In [ ]:
# [4]> Pre-compute angles offset and add
# --------------------------------------------------------------------------------------------------------------
if conf["offset"] == "auto":
    offset_angles, _shooting_pts_offset_selection = auto_angles_offset_pre_computation(flight_data_dict, shooting_pts_list, configuration=conf)
    add_offset_theoretical_angles(shooting_pts_list, offset=offset_angles)
    print(shooting_pts_list[0])

In [ ]:
# 5 > Selection of frames
# --------------------------------------------------------------------------------------------------------------    
shooting_pts_selection = select_matching_pairs(flight_data_dict, shooting_pts_list, configuration=conf, selection_option=conf.get("selection", "all"))
print(f"selectected {len(shooting_pts_selection)}/{len(shooting_pts_list)} image pairs")

In [ ]:
# 6 > Image processing.
# --------------------------------------------------------------------------------------------------------------
matched_images_paths_selection = list(map(lambda pt: (pt.Vis, pt.Nir), shooting_pts_selection))
image_processing_engine(matched_images_paths_selection, shooting_pts_selection, odm_multispectral=conf["odm_multispectral"], traces=conf["traces"], configuration=conf, skip=True)
print(shooting_pts_selection[0])

In [ ]:
# 8 > Analysis
# --------------------------------------------------------------------------------------------------------------
    
# analyzis(flight_data_dict, shooting_pts_selection, configuration=conf)
analyzis(flight_data_dict, shooting_pts_list, configuration=conf)